# TF Serving工作流程

![img](./img/tf2.png)

TF Serving的工作流程主要分为以下几个步骤：

- Source会针对需要进行加载的模型创建一个Loader，Loader中会包含要加载模型的全部信息；
- Source通知Manager有新的模型需要进行加载；
- Manager通过版本管理策略（Version Policy）来确定哪些模型需要被下架，哪些模型需要被加载；
- Manger在确认需要加载的模型符合加载策略，便通知Loader来加载最新的模型；
- 客户端像服务端请求模型结果时，可以指定模型的版本，也可以使用最新模型的结果；



#### 简单示例

TF Serving客户端和服务端的通信方式有两种（gRPC和RESTfull API）

##### 我这里用的是RESTfull API形式

- **1. 准备TF Serving的Docker环境**

目前TF Serving有Docker、APT（二级制安装）和源码编译三种方式，我使用使用Docker方式。

```shell
docker pull tensorflow/serving
```

- **2. 下载官方示例代码**

示例代码中包含已训练好的模型和与服务端进行通信的客户端（RESTfull API形式不需要专门的客户端）

```shell
mkdir -p /tmp/tfserving
cd /tmp/tfserving
git clone https://github.com/tensorflow/serving
```

- **3. 运行TF Serving**

```shell
docker run -p 8501:8501 \
  --mount type=bind,\
   source=/tmp/tfserving/serving/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_two_cpu,\
target=/models/half_plus_two \
-e MODEL_NAME=half_plus_two -t tensorflow/serving &
```

这里需要注意的是，较早的docker版本没有“--mount”选项，比如Ubuntu16.04默认安装的docker就没有（我的环境是Ubuntu 18.04）。

- **4.客户端验证**

```shell
curl -d '{"instances": [1.0, 2.0, 5.0]}' \
  -X POST http://localhost:8501/v1/models/half_plus_two:predict 
```

返回结果，

```shell
{ "predictions": [2.5, 3.0, 4.5] }
```

![img](./img/tf1.png)